In [18]:
import pandas as pd
import cufflinks as cf
import math
import seaborn as sns
import numpy as np
from scipy.spatial import distance

# データロード

In [88]:
#データロード
columns = ["treat","age","educ","black","hispan","married","nodegree","re75","re78"]
treated = pd.read_csv('input/nsw_treated.txt',header=None,delimiter='  ')
treated.columns = columns
print(treated.shape)
treated.head()

(297, 9)


/home/watanabe/.local/share/virtualenvs/causal_inference_lalomde-bTRS2FO0/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,treat,age,educ,black,hispan,married,nodegree,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.7899


In [89]:
columns = ["treat","age","educ","black","hispan","married","nodegree","re75","re78"]
controlled = pd.read_csv('input/nsw_control.txt',header=None,delimiter='  ')
controlled.columns = columns
print(controlled.shape)
controlled.head()

(425, 9)


/home/watanabe/.local/share/virtualenvs/causal_inference_lalomde-bTRS2FO0/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,treat,age,educ,black,hispan,married,nodegree,re75,re78
0,0.0,23.0,10.0,1.0,0.0,0.0,1.0,0.000,0.00
1,0.0,26.0,12.0,0.0,0.0,0.0,0.0,0.000,12383.68
2,0.0,22.0,9.0,1.0,0.0,0.0,1.0,0.000,0.00
3,0.0,34.0,9.0,1.0,0.0,0.0,1.0,4368.413,14051.16
4,0.0,18.0,9.0,1.0,0.0,0.0,1.0,0.000,10740.08


In [90]:
all_sample = pd.concat((treated,controlled), axis=0)
all_sample.shape

(722, 9)

# データ確認

## 施策の効果の推定

In [5]:
# re78のtreatの条件付き平均値の差
cf.go_offline()
all_sample.groupby('treat').mean()['re78'].iplot(kind='bar')

というわけでこの施策の効果は1000位です  

## 反論１：黒人は収入が少ない.この平均値の差は黒人かどうかによるものだ

In [6]:
all_sample.groupby('treat').mean()['black'].iplot(kind='bar')

２つのグループで黒人の数は釣り合っている。よって、反論1は却下できる。

## 反論２：教育を受けた年数が長ければ収入が増える。この平均値の差は教育を受けた年数によるものだ

In [7]:
all_sample.groupby('treat').mean()['educ'].iplot(kind='bar')

うーん少しは異なっているかも。。。

## 反論3:学位を取ると収入が高くなる。施策を受けたグループには学位を取っていない人が施策を受けていないグループより少ないから、施策の効果は単純な平均値の差より小さいはずだ。

In [8]:
all_sample.groupby('treat').mean()['nodegree'].iplot(kind='bar')

あるかもね

答えるべき問題：グループ間で変数は偏っているのか？

# グループ間で変数が偏っているかどうかの指標：SMD(Standardized difference)

グループ間で変数が偏っていないことをbalanceしていると呼ぶ。balanceしているかどうかを判断する指標として、SMDが用いられる。

In [97]:
def caliculate_SMD(treated_x, controlled_x):
    return abs((treated_x.mean()-controlled_x.mean()) / math.sqrt((treated_x.std()**2 + controlled_x.std()**2)/2))

In [100]:
def print_SMD(all_sample):
    for column_name, item in all_sample.iteritems():
        print(column_name)
        print('SMD is ',caliculate_SMD(all_sample.loc[all_sample['treat'] == 1,column_name], all_sample.loc[all_sample['treat'] == 0,column_name]))

In [101]:
print_SMD(all_sample)

treat
SMD is  inf
age
SMD is  0.02699459214545177
educ
SMD is  0.11169514216793087
black
SMD is  0.0033664496935312455
hispan
SMD is  0.0611890422867755
married
SMD is  0.02893847313003525
nodegree
SMD is  0.1997910779459351
re75
SMD is  0.007819420948937669
re78
SMD is  0.13958361580954365


/home/watanabe/.local/share/virtualenvs/causal_inference_lalomde-bTRS2FO0/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in double_scalars



0.1ならok  
0.1~0.2ならセーフ  
0.2以上ならbalanceしてないと解釈する（らしい）

学位の有る無しがbalanceしてないと判断する

# どうやってバランスさせるか：マッチング  
マッチング前後でSMDの差を確認する

## マハラノビス距離に基づくマッチング

    1:介入群のサンプルをランダムに並び替える
    2:介入群の一番上のサンプルと、対象群のすべてのサンプルのマハラノビス距離を計算し、最小のペアを見つける。
    3:そのペアを削除する。
    4:介入群のサンプルがなくなるまで2,3を繰り返す。

In [91]:
def match_mahalabinos(treated_data, controlled_data):
    pairs = []
    treated = treated_data.copy()
    controlled = controlled_data.copy()
    for treated_index, treated_sample in treated.iterrows():

        dist_list = []

        for controlled_index ,controlled_sample in controlled.iterrows():
            dist = distance.mahalanobis(treated_sample , controlled_sample , cov)
            dist_list.append(dist)
        
#         print(controlled.shape)
#         print(dist_list.index(min(dist_list)))
        controlled_min_index = dist_list.index(min(dist_list))
        pair = {'treated_index':treated_index, 'controlled_index':controlled.index[controlled_min_index]}
        pairs.append(pair)

        controlled = controlled.drop(controlled.index[controlled_min_index])
        
    return pairs

    


In [92]:
pairs = match_mahalabinos(treated, controlled)

(425, 9)
33
(424, 9)
402
(423, 9)
124
(422, 9)
188
(421, 9)
226
(420, 9)
362
(419, 9)
108
(418, 9)
406
(417, 9)
189
(416, 9)
96
(415, 9)
235
(414, 9)
47
(413, 9)
165
(412, 9)
372
(411, 9)
160
(410, 9)
374
(409, 9)
114
(408, 9)
31
(407, 9)
328
(406, 9)
4
(405, 9)
205
(404, 9)
400
(403, 9)
77
(402, 9)
69
(401, 9)
189
(400, 9)
128
(399, 9)
318
(398, 9)
38
(397, 9)
175
(396, 9)
30
(395, 9)
73
(394, 9)
345
(393, 9)
161
(392, 9)
110
(391, 9)
298
(390, 9)
60
(389, 9)
32
(388, 9)
97
(387, 9)
53
(386, 9)
48
(385, 9)
299
(384, 9)
110
(383, 9)
171
(382, 9)
102
(381, 9)
77
(380, 9)
327
(379, 9)
104
(378, 9)
58
(377, 9)
351
(376, 9)
343
(375, 9)
1
(374, 9)
236
(373, 9)
21
(372, 9)
78
(371, 9)
359
(370, 9)
358
(369, 9)
54
(368, 9)
323
(367, 9)
359
(366, 9)
51
(365, 9)
351
(364, 9)
181
(363, 9)
327
(362, 9)
120
(361, 9)
302
(360, 9)
294
(359, 9)
358
(358, 9)
106
(357, 9)
6
(356, 9)
243
(355, 9)
0
(354, 9)
290
(353, 9)
52
(352, 9)
21
(351, 9)
333
(350, 9)
310
(349, 9)
2
(348, 9)
2
(347, 9)
162
(346, 9

In [96]:
matched_index = [x['controlled_index'] for x in pairs]
matched_controlled = controlled.loc[matced_index ,:]
print(matched_controlled.shape)
matched_controlled.head()

(297, 9)


,treat,age,educ,black,hispan,married,nodegree,re75,re78
33,0.0,22.0,8.0,0.0,1.0,0.0,1.0,0.0,9920.9450
403,0.0,17.0,9.0,1.0,0.0,0.0,1.0,0.0,3590.7020
125,0.0,25.0,10.0,1.0,0.0,0.0,1.0,0.0,20942.2400
190,0.0,27.0,13.0,1.0,0.0,0.0,0.0,0.0,7609.5180
229,0.0,25.0,10.0,1.0,0.0,0.0,1.0,0.0,289.7899


In [105]:
treated

,treat,age,educ,black,hispan,married,nodegree,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000,289.7899
...,...,...,...,...,...,...,...,...,...
292,1.0,20.0,9.0,1.0,0.0,0.0,1.0,0.000,8881.6650
293,1.0,31.0,4.0,1.0,0.0,0.0,1.0,4023.211,7382.5490
294,1.0,24.0,10.0,1.0,0.0,1.0,1.0,4078.152,0.0000
295,1.0,33.0,11.0,1.0,0.0,1.0,1.0,25142.240,4181.9420


In [108]:
all_matched_sample = pd.concat((treated, matched_controlled), axis=0)
print_SMD(all_matched_sample)

treat
SMD is  inf
age
SMD is  0.09311634140213589
educ
SMD is  0.09600669296658523
black
SMD is  0.041503719909505375
hispan
SMD is  0.09706736316253686
married
SMD is  0.05530189354470355
nodegree
SMD is  0.20161587371660813
re75
SMD is  0.002798651102150844
re78
SMD is  0.04177022542075894


/home/watanabe/.local/share/virtualenvs/causal_inference_lalomde-bTRS2FO0/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in double_scalars



In [122]:
def plot_SMDplot(before_matched_data, after_matched_data):
    before_SMD = []
    after_SMD = []
    drop_columns = ['re78']
    before_matched_data = before_matched_data.drop(drop_columns,axis=1)
    after_matched_data = after_matched_data.drop(drop_columns,axis=1)
    
    for column_name, item in before_matched_data.iteritems():
        SMD = caliculate_SMD(before_matched_data.loc[before_matched_data['treat'] == 1,column_name], before_matched_data.loc[before_matched_data['treat'] == 0,column_name])
        before_SMD.append(SMD)
 
    for column_name, item in after_matched_data.iteritems():
        SMD = caliculate_SMD(after_matched_data.loc[after_matched_data['treat'] == 1,column_name], after_matched_data.loc[after_matched_data['treat'] == 0,column_name])
        after_SMD.append(SMD)
    
    
    SMD_pdf = pd.DataFrame(data=(before_SMD,after_SMD), index=['before_matcheing','after_matching'], columns=after_matched_data.columns)
    return SMD_pdf

In [123]:
plot_SMDplot(all_sample, all_matched_sample)

/home/watanabe/.local/share/virtualenvs/causal_inference_lalomde-bTRS2FO0/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in double_scalars



,treat,age,educ,black,hispan,married,nodegree,re75
before_matcheing,inf,0.026995,0.111695,0.003366,0.061189,0.028938,0.199791,0.007819
after_matching,inf,0.093116,0.096007,0.041504,0.097067,0.055302,0.201616,0.002799
